In this section, we are experimenting with OSRM and different routing techniques to create efficient routing mechanism. 

The most common use of OSRM is to read location coordinate from a CSV file such as locations.csv and create routing between these locations. Please see a sample of location.csv in Ethiopia:

name,region,country,latitude,longitude,location_type,conflict_date,population
Addis Ababa,Addis Ababa,Ethiopia,8.978098143949728,38.75857450155794,town,0,0
Mekele,Mekele,Ethiopia,13.495486756898567,39.46589089154361,town,0,0
Dire Dawa,Dire Dawa,Ethiopia,9.602835645348511,41.85443640082678,town,0,0
Adama,Adama,Ethiopia,8.526471189559459,39.25963003366467,town,0,0

Install packages that will be used. 

In [ ]:
import sys
!{sys.executable} -m pip install pandas folium polyline tsp_solver

In [ ]:
import os
import pandas as pd
import requests
import folium
import polyline

# Set the file name of the locations.csv file
locations_csv = 'ethiopia-locations.csv'

# Set the path to the directory where route.geojson is stored
locations_csv_path = '.'

# Check if the CSV file exists
locations_csv_file = os.path.join(locations_csv_path, locations_csv)
if not os.path.exists(locations_csv_file):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Read the locations from CSV
df = pd.read_csv(locations_csv_file)

# Create a map centered around the first location
start_lat = df.loc[0, 'latitude']
start_lon = df.loc[0, 'longitude']
m = folium.Map(location=[start_lat, start_lon], zoom_start=6)

# Create an empty list to store route instructions
route_instructions = []

# Iterate through each pair of locations in the DataFrame
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        # Extract location details
        name1 = df.loc[i, 'name']
        lat1 = df.loc[i, 'latitude']
        lon1 = df.loc[i, 'longitude']
        name2 = df.loc[j, 'name']
        lat2 = df.loc[j, 'latitude']
        lon2 = df.loc[j, 'longitude']

        # Prepare API request URL
        osrm_endpoint = 'http://localhost:5000/route/v1/driving'
        request_url = f"{osrm_endpoint}/{lon1},{lat1};{lon2},{lat2}?steps=true&geometries=polyline"

        try:
            # Send request to OSRM
            response = requests.get(request_url)
            response.raise_for_status()

            # Extract route data from the response
            route_data = response.json()

            # Check if response contains a route
            if 'routes' in route_data and len(route_data['routes']) > 0:
                # Extract route steps
                steps = route_data['routes'][0]['legs'][0]['steps']
                distance_sum = 0
                for step in steps:
                    # Extract step instructions
                    instruction = step['name']

                    # Extract step distance and add to the distance sum
                    distance = step['distance']
                    distance_sum += distance

                    # Append step information to the route_instructions list
                    route_instructions.append((instruction, distance_sum))

                # Extract route geometry
                geometry = route_data['routes'][0]['geometry']

                # Decode polyline string to get coordinates
                coordinates = polyline.decode(geometry)

                # Create a Folium PolyLine object
                polyline_obj = folium.PolyLine(
                    locations=coordinates,
                    color='blue',
                    weight=2,
                    opacity=0.9
                )

                # Add the PolyLine to the map
                polyline_obj.add_to(m)

        except requests.exceptions.RequestException as e:
            print(f"Error occurred during OSRM API request: {e}")
            continue

        # Add markers for the start and end locations
        folium.CircleMarker(
            location=[lat1, lon1],
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name1
        ).add_to(m)

        folium.CircleMarker(
            location=[lat2, lon2],
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name2
        ).add_to(m)

# Save the map as an HTML file
map_file = os.path.join('images', 'ethiopia-route-map.html')
m.save(map_file)
print(f"Map saved as {map_file}")

# Display the map
m


In [ ]:
import os
import pandas as pd
import requests
import folium
import polyline

# Set the file name of the locations.csv file
locations_csv = 'ethiopia-locations.csv'

# Set the path to the directory where route.geojson is stored
locations_csv_path = '.'

# Check if the CSV file exists
locations_csv_file = os.path.join(locations_csv_path, locations_csv)
if not os.path.exists(locations_csv_file):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Read the locations from CSV
df = pd.read_csv(locations_csv_file)

# Create a map centered around the first location
start_lat = df.loc[0, 'latitude']
start_lon = df.loc[0, 'longitude']
m = folium.Map(location=[start_lat, start_lon], zoom_start=6)

# Create an empty set to store processed route pairs
processed_pairs = set()

# Iterate through each pair of locations in the DataFrame
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        # Extract location details
        name1 = df.loc[i, 'name']
        lat1 = df.loc[i, 'latitude']
        lon1 = df.loc[i, 'longitude']
        name2 = df.loc[j, 'name']
        lat2 = df.loc[j, 'latitude']
        lon2 = df.loc[j, 'longitude']

        # Check if the reverse pair already exists in the processed pairs set
        if (name2, name1) in processed_pairs:
            continue

        # Add the pair to the processed pairs set
        processed_pairs.add((name1, name2))

        # Prepare API request URL
        osrm_endpoint = 'http://localhost:5000/route/v1/driving'
        request_url = f"{osrm_endpoint}/{lon1},{lat1};{lon2},{lat2}?steps=true&geometries=polyline"

        try:
            # Send request to OSRM
            response = requests.get(request_url)
            response.raise_for_status()

            # Extract route data from the response
            route_data = response.json()

            # Check if response contains a route
            if 'routes' in route_data and len(route_data['routes']) > 0:
                # Extract route steps
                steps = route_data['routes'][0]['legs'][0]['steps']
                distance_sum = 0
                for step in steps:
                    # Extract step instructions
                    instruction = step['name']

                    # Extract step distance and add to the distance sum
                    distance = step['distance']
                    distance_sum += distance

                # Append step information to the route_instructions list
                route_instructions.append((instruction, distance_sum))

                # Extract route geometry
                geometry = route_data['routes'][0]['geometry']

                # Decode polyline string to get coordinates
                coordinates = polyline.decode(geometry)

                # Create a Folium PolyLine object
                polyline_obj = folium.PolyLine(
                    locations=coordinates,
                    color='blue',
                    weight=2,
                    opacity=0.9
                )

                # Add the PolyLine to the map
                polyline_obj.add_to(m)

        except requests.exceptions.RequestException as e:
            print(f"Error occurred during OSRM API request: {e}")
            continue

        # Add markers for the start and end locations
        folium.CircleMarker(
            location=[lat1, lon1],
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name1
        ).add_to(m)

        folium.CircleMarker(
            location=[lat2, lon2],
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name2
        ).add_to(m)

# Save the map as an HTML file
map_file = os.path.join('images', 'ethiopia-route-map.html')
m.save(map_file)
print(f"Map saved as {map_file}")

# Display the map
m

In [ ]:
import os
import pandas as pd
import requests
import folium
import polyline

# Set the file name of the locations.csv file
locations_csv = 'ethiopia-locations.csv'

# Set the path to the directory where route.geojson is stored
locations_csv_path = '.'

# Check if the CSV file exists
locations_csv_file = os.path.join(locations_csv_path, locations_csv)
if not os.path.exists(locations_csv_file):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Read the locations from CSV
df = pd.read_csv(locations_csv_file)

# Create a map centered around the first location
start_lat = df.loc[0, 'latitude']
start_lon = df.loc[0, 'longitude']
m = folium.Map(location=[start_lat, start_lon], zoom_start=6)

# Create an empty set to store processed route pairs
processed_pairs = set()

# Iterate through each pair of locations in the DataFrame
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        # Extract location details
        name1 = df.loc[i, 'name']
        lat1 = df.loc[i, 'latitude']
        lon1 = df.loc[i, 'longitude']
        name2 = df.loc[j, 'name']
        lat2 = df.loc[j, 'latitude']
        lon2 = df.loc[j, 'longitude']

        # Create frozensets for the pair of locations
        location_pair = frozenset([name1, name2])

        # Check if the frozenset already exists in the processed pairs set
        if location_pair in processed_pairs:
            continue

        # Add the frozenset to the processed pairs set
        processed_pairs.add(location_pair)

        # Prepare API request URL
        osrm_endpoint = 'http://localhost:5000/route/v1/driving'
        request_url = f"{osrm_endpoint}/{lon1},{lat1};{lon2},{lat2}?steps=true&geometries=polyline"

        try:
            # Send request to OSRM
            response = requests.get(request_url)
            response.raise_for_status()

            # Extract route data from the response
            route_data = response.json()

            # Check if response contains a route
            if 'routes' in route_data and len(route_data['routes']) > 0:
                # Extract route steps
                steps = route_data['routes'][0]['legs'][0]['steps']
                distance_sum = 0
                for step in steps:
                    # Extract step instructions
                    instruction = step['name']

                    # Extract step distance and add to the distance sum
                    distance = step['distance']
                    distance_sum += distance

                # Append step information to the route_instructions list
                route_instructions.append((instruction, distance_sum))

                # Extract route geometry
                geometry = route_data['routes'][0]['geometry']

                # Decode polyline string to get coordinates
                coordinates = polyline.decode(geometry)

                # Create a Folium PolyLine object
                polyline_obj = folium.PolyLine(
                    locations=coordinates,
                    color='blue',
                    weight=2,
                    opacity=0.9
                )

                # Add the PolyLine to the map
                polyline_obj.add_to(m)

        except requests.exceptions.RequestException as e:
            print(f"Error occurred during OSRM API request: {e}")
            continue

        # Add markers for the start and end locations
        folium.CircleMarker(
            location=[lat1, lon1],
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name1
        ).add_to(m)

        folium.CircleMarker(
            location=[lat2, lon2],
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name2
        ).add_to(m)

# Save the map as an HTML file
map_file = os.path.join('images', 'ethiopia-route-map.html')
m.save(map_file)
print(f"Map saved as {map_file}")

# Display the map
m

In [ ]:
import os
import pandas as pd
import requests
import folium
import polyline
from sklearn.cluster import DBSCAN

# Set the file name of the locations.csv file
locations_csv = 'ethiopia-locations.csv'

# Set the path to the directory where route.geojson is stored
locations_csv_path = '.'

# Check if the CSV file exists
locations_csv_file = os.path.join(locations_csv_path, locations_csv)
if not os.path.exists(locations_csv_file):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Read the locations from CSV
df = pd.read_csv(locations_csv_file)

# Create a map centered around the first location
start_lat = df.loc[0, 'latitude']
start_lon = df.loc[0, 'longitude']
m = folium.Map(location=[start_lat, start_lon], zoom_start=6)

# Create an empty list to store route instructions
route_instructions = []

# Create a set to store visited location pairs
visited_pairs = set()

# Iterate through each pair of locations in the DataFrame
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        # Extract location details
        name1 = df.loc[i, 'name']
        lat1 = df.loc[i, 'latitude']
        lon1 = df.loc[i, 'longitude']
        name2 = df.loc[j, 'name']
        lat2 = df.loc[j, 'latitude']
        lon2 = df.loc[j, 'longitude']

        # Check if the pair has been visited before (or its reverse pair)
        if (name1, name2) in visited_pairs or (name2, name1) in visited_pairs:
            continue

        # Prepare API request URL
        osrm_endpoint = 'http://localhost:5000/route/v1/driving'
        request_url = f"{osrm_endpoint}/{lon1},{lat1};{lon2},{lat2}?steps=true&geometries=polyline"

        try:
            # Send request to OSRM
            response = requests.get(request_url)
            response.raise_for_status()

            # Extract route data from the response
            route_data = response.json()

            # Check if response contains a route
            if 'routes' in route_data and len(route_data['routes']) > 0:
                # Extract route steps
                steps = route_data['routes'][0]['legs'][0]['steps']
                distance_sum = 0
                for step in steps:
                    # Extract step instructions
                    instruction = step['name']

                    # Extract step distance and add to the distance sum
                    distance = step['distance']
                    distance_sum += distance

                # Append step information to the route_instructions list
                route_instructions.append((instruction, distance_sum))

                # Extract route geometry
                geometry = route_data['routes'][0]['geometry']

                # Decode polyline string to get coordinates
                coordinates = polyline.decode(geometry)

                # Add the coordinates to the map as a PolyLine
                folium.PolyLine(
                    locations=coordinates,
                    color='blue',
                    weight=2,
                    opacity=0.9
                ).add_to(m)

                # Add the pair to the visited set and its reverse pair
                visited_pairs.add((name1, name2))
                visited_pairs.add((name2, name1))

        except requests.exceptions.RequestException as e:
            print(f"Error occurred during OSRM API request: {e}")
            continue

        # Add markers for the start and end locations
        folium.CircleMarker(
            location=[lat1, lon1],
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name1
        ).add_to(m)

        folium.CircleMarker(
            location=[lat2, lon2],
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name2
        ).add_to(m)

# Save the map as an HTML file
map_file = os.path.join('images', 'ethiopia-route-map.html')
m.save(map_file)
print(f"Map saved as {map_file}")

# Display the map
m

In [ ]:
import os
import pandas as pd
import requests
import folium
import polyline
from sklearn.cluster import DBSCAN

# Set the file name of the locations.csv file
locations_csv = 'ethiopia-locations.csv'

# Set the path to the directory where route.geojson is stored
locations_csv_path = '.'

# Check if the CSV file exists
locations_csv_file = os.path.join(locations_csv_path, locations_csv)
if not os.path.exists(locations_csv_file):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Read the locations from CSV
df = pd.read_csv(locations_csv_file)

# Create a map centered around the first location
start_lat = df.loc[0, 'latitude']
start_lon = df.loc[0, 'longitude']
m = folium.Map(location=[start_lat, start_lon], zoom_start=6)

# Create an empty list to store route instructions
route_instructions = []

# Iterate through the sequential pairs of locations in the DataFrame
for i in range(len(df)):
    # Extract location details
    name1 = df.loc[i, 'name']
    lat1 = df.loc[i, 'latitude']
    lon1 = df.loc[i, 'longitude']
    
    # Check if it is the last location
    if i == len(df) - 1:
        name2 = df.loc[0, 'name']
        lat2 = df.loc[0, 'latitude']
        lon2 = df.loc[0, 'longitude']
    else:
        name2 = df.loc[i + 1, 'name']
        lat2 = df.loc[i + 1, 'latitude']
        lon2 = df.loc[i + 1, 'longitude']

    # Prepare API request URL
    osrm_endpoint = 'http://localhost:5000/route/v1/driving'
    request_url = f"{osrm_endpoint}/{lon1},{lat1};{lon2},{lat2}?steps=true&geometries=polyline"

    try:
        # Send request to OSRM
        response = requests.get(request_url)
        response.raise_for_status()

        # Extract route data from the response
        route_data = response.json()

        # Check if response contains a route
        if 'routes' in route_data and len(route_data['routes']) > 0:
            # Extract route steps
            steps = route_data['routes'][0]['legs'][0]['steps']
            distance_sum = 0
            for step in steps:
                # Extract step instructions
                instruction = step['name']

                # Extract step distance and add to the distance sum
                distance = step['distance']
                distance_sum += distance

            # Append step information to the route_instructions list
            route_instructions.append((instruction, distance_sum))

            # Extract route geometry
            geometry = route_data['routes'][0]['geometry']

            # Decode polyline string to get coordinates
            coordinates = polyline.decode(geometry)

            # Add the coordinates to the map as a PolyLine
            folium.PolyLine(
                locations=coordinates,
                color='blue',
                weight=2,
                opacity=0.9
            ).add_to(m)

        # Add markers for the start and end locations
        folium.CircleMarker(
            location=[lat1, lon1],
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name1
        ).add_to(m)

        folium.CircleMarker(
            location=[lat2, lon2],
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name2
        ).add_to(m)

    except requests.exceptions.RequestException as e:
        print(f"Error occurred during OSRM API request: {e}")
        continue

# Save the map as an HTML file
map_file = os.path.join('images', 'ethiopia-route-map.html')
m.save(map_file)
print(f"Map saved as {map_file}")

# Display the map
m

In [ ]:
import os
import pandas as pd
import requests
import folium
import polyline
from tsp_solver.greedy import solve_tsp

# Set the file name of the locations.csv file
locations_csv = 'ethiopia-locations.csv'

# Set the path to the directory where route.geojson is stored
locations_csv_path = '.'

# Check if the CSV file exists
locations_csv_file = os.path.join(locations_csv_path, locations_csv)
if not os.path.exists(locations_csv_file):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Read the locations from CSV
df = pd.read_csv(locations_csv_file)

# Create a map centered around the first location
start_lat = df.loc[0, 'latitude']
start_lon = df.loc[0, 'longitude']
m = folium.Map(location=[start_lat, start_lon], zoom_start=6)

# Create an empty list to store route instructions
route_instructions = []

# Create an empty list to store coordinates for clustering
all_coordinates = []

# Iterate through each pair of locations in the DataFrame
for i in range(len(df)):
    for j in range(len(df)):
        if i != j:
            # Extract location details
            name1 = df.loc[i, 'name']
            lat1 = df.loc[i, 'latitude']
            lon1 = df.loc[i, 'longitude']
            name2 = df.loc[j, 'name']
            lat2 = df.loc[j, 'latitude']
            lon2 = df.loc[j, 'longitude']

            # Prepare API request URL
            osrm_endpoint = 'http://localhost:5000/route/v1/driving'
            request_url = f"{osrm_endpoint}/{lon1},{lat1};{lon2},{lat2}?steps=true&geometries=polyline"

            try:
                # Send request to OSRM
                response = requests.get(request_url)
                response.raise_for_status()

                # Extract route data from the response
                route_data = response.json()

                # Check if response contains a route
                if 'routes' in route_data and len(route_data['routes']) > 0:
                    # Extract route steps
                    steps = route_data['routes'][0]['legs'][0]['steps']
                    distance_sum = 0
                    for step in steps:
                        # Extract step instructions
                        instruction = step['name']

                        # Extract step distance and add to the distance sum
                        distance = step['distance']
                        distance_sum += distance

                    # Append step information to the route_instructions list
                    route_instructions.append((instruction, distance_sum))

                    # Extract route geometry
                    geometry = route_data['routes'][0]['geometry']

                    # Decode polyline string to get coordinates
                    coordinates = polyline.decode(geometry)

                    # Add the coordinates to the list for clustering
                    all_coordinates.extend(coordinates)

            except requests.exceptions.RequestException as e:
                print(f"Error occurred during OSRM API request: {e}")
                continue

            # Add markers for the start and end locations
            folium.CircleMarker(
                location=[lat1, lon1],
                radius=6,
                color='red',
                fill=True,
                fill_color='blue',
                popup=name1
            ).add_to(m)

            folium.CircleMarker(
                location=[lat2, lon2],
                radius=6,
                color='red',
                fill=True,
                fill_color='blue',
                popup=name2
            ).add_to(m)

# Create a distance matrix for the TSP solver
n = len(all_coordinates)
dist_matrix = [[0] * n for _ in range(n)]
for i in range(n):
    for j in range(n):
        # Calculate the distance between coordinates i and j
        dist_matrix[i][j] = ((all_coordinates[i][0] - all_coordinates[j][0]) ** 2 +
                            (all_coordinates[i][1] - all_coordinates[j][1]) ** 2) ** 0.5

# Solve the TSP using the greedy solver
tsp_route = solve_tsp(dist_matrix)

# Reorder the coordinates based on the TSP route
reordered_coordinates = [all_coordinates[i] for i in tsp_route]

# Create a Folium PolyLine object for the TSP route
polyline_obj = folium.PolyLine(
    locations=reordered_coordinates,
    color='blue',
    weight=2,
    opacity=0.9
)

# Add the TSP route to the map
polyline_obj.add_to(m)

# Save the map as an HTML file
map_file = os.path.join('images', 'ethiopia-route-map.html')
m.save(map_file)
print(f"Map saved as {map_file}")

# Display the map
m

In [ ]:
import os
import pandas as pd
import requests
import folium
import polyline
from sklearn.cluster import DBSCAN
from scipy.spatial import distance_matrix
from scipy.optimize import minimize

# Set the file name of the locations.csv file
locations_csv = 'ethiopia-locations.csv'

# Set the path to the directory where route.geojson is stored
locations_csv_path = '.'

# Check if the CSV file exists
locations_csv_file = os.path.join(locations_csv_path, locations_csv)
if not os.path.exists(locations_csv_file):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Read the locations from CSV
df = pd.read_csv(locations_csv_file)

# Create a map centered around the first location
start_lat = df.loc[0, 'latitude']
start_lon = df.loc[0, 'longitude']
m = folium.Map(location=[start_lat, start_lon], zoom_start=6)

# Create an empty list to store route instructions
route_instructions = []

# Create an empty list to store coordinates for clustering
all_coordinates = []

# Iterate through each pair of locations in the DataFrame
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        # Extract location details
        name1 = df.loc[i, 'name']
        lat1 = df.loc[i, 'latitude']
        lon1 = df.loc[i, 'longitude']
        name2 = df.loc[j, 'name']
        lat2 = df.loc[j, 'latitude']
        lon2 = df.loc[j, 'longitude']

        # Prepare API request URL
        osrm_endpoint = 'http://localhost:5000/route/v1/driving'
        request_url = f"{osrm_endpoint}/{lon1},{lat1};{lon2},{lat2}?steps=true&geometries=polyline"

        try:
            # Send request to OSRM
            response = requests.get(request_url)
            response.raise_for_status()

            # Extract route data from the response
            route_data = response.json()

            # Check if response contains a route
            if 'routes' in route_data and len(route_data['routes']) > 0:
                # Extract route steps
                steps = route_data['routes'][0]['legs'][0]['steps']
                distance_sum = 0
                for step in steps:
                    # Extract step instructions
                    instruction = step['name']

                    # Extract step distance and add to the distance sum
                    distance = step['distance']
                    distance_sum += distance

                # Append step information to the route_instructions list
                route_instructions.append((instruction, distance_sum))

                # Extract route geometry
                geometry = route_data['routes'][0]['geometry']

                # Decode polyline string to get coordinates
                coordinates = polyline.decode(geometry)

                # Add the coordinates to the list for clustering
                all_coordinates.extend(coordinates)

                # Add markers for the start and end locations
                folium.CircleMarker(
                    location=[lat1, lon1],
                    radius=6,
                    color='red',
                    fill=True,
                    fill_color='blue',
                    popup=name1
                ).add_to(m)

                folium.CircleMarker(
                    location=[lat2, lon2],
                    radius=6,
                    color='red',
                    fill=True,
                    fill_color='blue',
                    popup=name2
                ).add_to(m)

        except requests.exceptions.RequestException as e:
            print(f"Error occurred during OSRM API request: {e}")
            continue

# Perform DBSCAN (Density-Based Spatial Clustering of Applications with Noise) for coordinates
eps = 0.01  # distance threshold for clustering
min_samples = 2  # minimum number of points in a cluster
dbscan = DBSCAN(eps=eps, min_samples=min_samples)
cluster_labels = dbscan.fit_predict(all_coordinates)

# Create a dictionary to store polylines for each cluster
cluster_polylines = {}

# Iterate over the clusters and generate polylines for each cluster
for label in set(cluster_labels):
    if label == -1:
        # Skip noise points (not assigned to any cluster)
        continue

    # Extract coordinates for the current cluster
    cluster_coords = [coord for i, coord in enumerate(all_coordinates) if cluster_labels[i] == label]

    # Calculate the distance matrix for the cluster coordinates
    distance_matrix_cluster = distance_matrix(cluster_coords, cluster_coords)

    # Define the objective function for TSP
    def tsp_objective_function(order):
        return sum([distance_matrix_cluster[order[i]][order[i + 1]] for i in range(len(order) - 1)])

    # Define the constraint function for TSP
    def tsp_constraint_function(order):
        return sum(order) - len(order)

    # Define the initial guess for the TSP
    initial_guess = list(range(len(cluster_coords)))

    # Define the bounds for the TSP
    bounds = [(0, len(cluster_coords) - 1)] * len(cluster_coords)

    # Define the constraint for the TSP
    constraint = {'type': 'eq', 'fun': tsp_constraint_function}

    # Solve the TSP using the minimize function
    result = minimize(tsp_objective_function, initial_guess, method='SLSQP', bounds=bounds, constraints=constraint)

    # Get the optimal order of locations from the result and convert to integers
    optimal_order = result.x.astype(int)

    # Reorder the cluster coordinates based on the optimal order
    reordered_coords = [cluster_coords[i] for i in optimal_order]

    # Create a list of polyline coordinates
    polyline_coords = [(coord[0], coord[1]) for coord in reordered_coords]

    # Create a Folium PolyLine object for the current cluster
    polyline_obj = folium.PolyLine(
        locations=polyline_coords,
        color='blue',
        weight=2,
        opacity=0.9
    )

    # Add the polyline to the map
    polyline_obj.add_to(m)

# Save the map as an HTML file
map_file = os.path.join('images', 'ethiopia-route-map.html')
m.save(map_file)
print(f"Map saved as {map_file}")

# Display the map
m